In [94]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [95]:
df = pd.read_csv('Data_tokenized.csv')
df.head()

,topic,tokens
0,Pháp luật,tên cướp tiệm vàng huế đại_úy công_an công_tác...
1,Sức khỏe - Đời sống,bỏ mạng 5 g nga tiến thẳng 4 g 6 g gần đây thứ...
2,Giáo dục,địa_phương đứng đầu tổng 3 môn văn_toán ngoại_...
3,Thế giới,người chết mưa_lũ nghìn một mỹ 28 thống_đốc ke...
4,Thời sự,hải_phòng hình_ảnh xe điên tai_nạn liên_hoàn p...


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161216 entries, 0 to 161215
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   topic   161216 non-null  object
 1   tokens  161216 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [97]:
tokenizer = Tokenizer(num_words=30000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['tokens'])


In [98]:
sequence = tokenizer.texts_to_sequences(df['tokens'])

In [99]:
print("Chuỗi số (sequences) - 5 câu đầu tiên:")
for i, seq in enumerate(sequence[:5]):
    print(f"Câu {i+1}: {seq}")

Chuỗi số (sequences) - 5 câu đầu tiên:
Câu 1: [1711, 1569, 1829, 381, 1201, 67, 2506, 2, 43, 2, 68, 1376, 1302, 386, 953, 35, 2, 43, 88, 745, 616, 1201, 20, 119, 6, 36, 1026, 1017, 1569, 1829, 381, 1041, 225, 650, 533, 82, 451, 974, 132, 143, 1201, 88, 745, 616, 1201, 20, 1274, 1319, 62, 680, 369, 1450, 953, 35, 30, 240, 96, 38, 1017, 5844, 156, 577, 2457, 1829, 381, 341, 306, 230, 236, 1801, 1041, 225, 650, 1026, 1017, 616, 78, 41, 1200, 2070, 42, 1829, 1569, 381, 157, 28, 30, 240, 381, 1569, 2973, 2570, 626, 40, 104, 148, 45, 12, 32, 104, 148, 86, 1036, 736, 284, 420, 2, 43, 88, 745, 616, 1201, 205, 669, 285, 41, 132, 66, 53, 57, 2, 43, 88, 2, 43, 143, 1201, 719, 114, 5, 18, 92, 177, 4, 741, 102, 64, 36, 4014, 811, 37, 40, 5, 18, 19, 1055, 766, 107, 1041, 225, 650, 37, 45, 12, 32, 162, 45, 477, 5, 18, 422, 419, 81, 70, 1171, 714, 176, 97, 381, 2105, 82, 1105, 2576, 138, 13473, 6673, 1417, 1041, 503, 104, 148, 25, 93, 143, 1201, 225, 37, 327, 77, 43, 84, 37, 66, 2, 43, 231, 30, 240, 8

In [100]:
oov_count = sum(seq.count(1) for seq in sequence)  # Index 1 là <OOV>
total_words = sum(len(seq) for seq in sequence)
print(f"Tỷ lệ OOV: {oov_count / total_words * 100:.2f}%")

Tỷ lệ OOV: 0.37%


In [101]:
df["token_length"] = df['tokens'].apply(lambda x: len(x.split()))
print(df["token_length"].describe())

count    161216.00000
mean        273.74456
std         189.84190
min           1.00000
25%         162.00000
50%         236.00000
75%         349.00000
max       34570.00000
Name: token_length, dtype: float64


In [102]:
maxlen = int(df['token_length'].quantile(0.95))  # Lấy bách phân vị 95
print(f"maxlen được chọn: {maxlen}")

maxlen được chọn: 572


In [103]:
X = pad_sequences(sequence, maxlen=maxlen, padding='post', truncating='post')
y = df["topic"]

In [104]:
merge1 = {
    "Quốc phòng": "Chính trị",
    "Bất động sản": "Kinh doanh - Tài chính",
    "Bạn đọc": "Xã hội",
    "Kinh tế": "Kinh doanh - Tài chính",
    "Thời sự": "Xã hội"
}

df["topic"] = df["topic"].replace(merge1)

# Sau đó gộp “Chính trị” vào “Thế giới”
df["topic"] = df["topic"].replace({"Chính trị": "Thế giới"})

df["topic"].value_counts()


topic
Xã hội                    32500
Sức khỏe - Đời sống       23241
Thế giới                  21639
Văn hóa - Giải trí        21224
Thể thao                  19263
Kinh doanh - Tài chính    17025
Pháp luật                 10650
Giáo dục                   7631
Công nghệ                  4248
Xe                         3795
Name: count, dtype: int64

In [105]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

RUS = RandomUnderSampler(sampling_strategy={'Xã hội': 20000, 'Sức khỏe - Đời sống': 20000, 'Thế giới': 20000, 'Văn hóa - Giải trí': 20000}, random_state=42)
X_RUS, y_RUS = RUS.fit_resample(X, y)
Smote = SMOTE(sampling_strategy={'Xe': 10000, 'Công nghệ':10000, 'Giáo dục':10000}, random_state=42)
X_smote, y_smote = Smote.fit_resample(X_RUS, y_RUS)
y_smote.value_counts()

ValueError: With under-sampling methods, the number of samples in a class should be less or equal to the original number of samples. Originally, there is 15580 samples and 20000 samples are asked.

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df["topic"])
y = to_categorical(y)
print("\nNhãn sau mã hóa (5 mẫu đầu):")
print(y[:5])
print(f"Kích thước nhãn: {y.shape}")
print(f"Danh sách topic: {le.classes_}")


Nhãn sau mã hóa (5 mẫu đầu):
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Kích thước nhãn: (161216, 10)
Danh sách topic: ['Công nghệ' 'Giáo dục' 'Kinh doanh - Tài chính' 'Pháp luật'
 'Sức khỏe - Đời sống' 'Thế giới' 'Thể thao' 'Văn hóa - Giải trí' 'Xe'
 'Xã hội']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42, stratify=y_smote)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D,Flatten, MaxPooling1D, Dense,Dropout

In [ ]:
model = Sequential([
    Embedding(30000, 100, input_length=maxlen),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Huấn luyện
history = model.fit(X_train, y_train, epochs=10, batch_size=64)

# Đánh giá
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nĐộ chính xác trên tập test: {test_acc:.4f}")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

ValueError: Invalid dtype: object